In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
from PIL import Image, ImageDraw
import numpy as np

from wct import Transfer

# Load models

In [ ]:
transfer = Transfer()

# Extract style

In [ ]:
style_image = Image.open('1280px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')
style_image

In [ ]:
%%time
style_features = transfer.get_features(np.array(style_image))
style_transforms = transfer.get_style(style_features)

# Load an image

In [ ]:
image = Image.open('deadpool.jpg')
image = image.resize((768, 448))
image

# Do style transfer

In [ ]:
content_features = transfer.get_features(np.array(image))

In [ ]:
%%time
X = 4
style_mean, coloring_matrix = style_transforms[X]
alpha = 0.5
colored_features = transfer.blend(content_features[X], X, style_mean, coloring_matrix, alpha)

# Decode

In [ ]:
%%time
restored_image = transfer.decode(colored_features, X)
restored_image = (np.clip(restored_image, 0, 255)).astype('uint8')

In [ ]:
Image.fromarray(restored_image)

# Do multi-level stylization

In [ ]:
%%time

restored_image = np.array(image)
for X in [5, 4, 3, 2, 1]:
    
    style_mean, coloring_matrix = style_transforms[X]
    alpha = 0.5

    content_features = transfer.get_features(restored_image, X)
    colored_features = transfer.blend(content_features, X, style_mean, coloring_matrix, alpha)

    restored_image = transfer.decode(colored_features, X)
    restored_image = (np.clip(restored_image, 0, 255)).astype('uint8')

In [ ]:
Image.fromarray(restored_image)